In [ ]:
"""
Script python pour ouvrir les fichiers de traces de clavier

"""

import matplotlib.pyplot as plt
import numpy as np
import time
import os


def read_int(f):
    ba = bytearray(4)
    f.readinto(ba)
    prm = np.frombuffer(ba, dtype=np.int32)
    return prm[0]


def read_double(f):
    ba = bytearray(8)
    f.readinto(ba)
    prm = np.frombuffer(ba, dtype=np.double)
    return prm[0]


def read_double_tab(f, n):
    ba = bytearray(8 * n)
    nr = f.readinto(ba)
    if nr != len(ba):
        return []
    else:
        prm = np.frombuffer(ba, dtype=np.double)
        return prm


def get_pics_from_file(filename):
    # Lecture du fichier d'infos + pics detectes (post-processing KeyFinder)
    f_pic = open(filename, "rb")
    info = dict()
    info["nb_pics"] = read_int(f_pic)
    info["freq_sampling_khz"] = read_double(f_pic)
    info["freq_trame_hz"] = read_double(f_pic)
    info["freq_pic_khz"] = read_double(f_pic)
    info["norm_fact"] = read_double(f_pic)
    tab_pics = []
    pics = read_double_tab(f_pic, info["nb_pics"])
    nb_trames = 1
    while len(pics) > 0:
        nb_trames = nb_trames + 1
        tab_pics.append(pics)
        pics = read_double_tab(f_pic, info["nb_pics"])
    f_pic.close()
    return tab_pics, info


def get_dataset():
    """
    returns the dataset in a dict with key : name of the key
    value: list of spikes
    """
    dataset = {}
    os.chdir('../data/')

    for filename in os.listdir():
        list_of_spikes, info = get_pics_from_file(filename)

        # we get the name of the key by removing "pics_" at the beginning and ".bin" at the end
        key = filename[5:-4]

        dataset[key] = list_of_spikes

    return dataset

In [ ]:
if __name__ == "__main__":
    pics_nokey, info = get_pics_from_file("../data/pics_NOKEY.bin")
    pics_pad0, info = get_pics_from_file("../data/pics_0.bin")
    
    print(len(pics_pad0))

    ######### Pics ############
    # NO KEY
    plt.figure(1)
    plt.subplot(211)
    plt.plot(range(1, info["nb_pics"] + 1), pics_nokey[0], 'ko')
    plt.xlabel('numéro de pic')
    plt.ylabel('valeur du pic')
    plt.title('no key')
    plt.ylim(0, 1.5)
    plt.grid(b=True, which='both')
    # PAD-0
    plt.subplot(212)
    plt.plot(range(1, info["nb_pics"] + 1), pics_pad0[0], 'ko')
    plt.xlabel('numéro de pic')
    plt.ylabel('valeur du pic')
    plt.title('PAD-0')
    plt.ylim(0, 1.5)
    plt.grid(b=True, which='both')
    #
    plt.show()
    
    


In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np

lr = LinearRegression()

dataset = get_dataset()
d_list = list(dataset)

X = []
Y = []
i = 0
for k in dataset:
    X += dataset[k]
    Y += [i] * len(dataset[k])
    i += 1

lr.fit(X, Y)



In [ ]:
x_pred, info = get_pics_from_file("../tohack/pics_LOGINMDP.bin")
#x_pred, info = get_pics_from_file("../data/pics_0.bin")

i_NOKEY = d_list.index('NOKEY')

res = []

for trame_pred in x_pred:
    t = [np.array(trame_pred)]
    predict_i = lr.predict(t)[0]
    predict_i = int(predict_i)
    if predict_i != i_NOKEY:
        res.append(d_list[predict_i])

print(res)

# print(len(res))
# tot_0 = 0
# for i in range(len(res)):
#     if res[i] == '3':
#         tot_0 += 1
# print(tot_0)

In [ ]:
for i_k in range(len(res)):
    if res[i_k] == 'CTRL' and res[i_k + 2] == 'SUPPR':
        print('##########')
        print(res[i_k:i_k + 22])
        


In [ ]:


d = get_dataset()

In [ ]:
print(len(d['1'][0]))

a = 0

for k in d:
    for i in range(len(d[k])):
        assert len(d[k][i]) == 17
        a += 1
        
print(a)

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

dataset = get_dataset()
d_list = list(dataset)

knn = KNeighborsClassifier(n_neighbors=len(dataset))

X = []
Y = []
i = 0
for k in dataset:
    X += dataset[k]
    Y += [i] * len(dataset[k])
    i += 1

knn.fit(X, Y)



In [ ]:
x_pred, info = get_pics_from_file("../tohack/pics_LOGINMDP.bin")
#x_pred, info = get_pics_from_file("../data/pics_0.bin")

i_NOKEY = d_list.index('NOKEY')

res = []

aa = 0
for trame_pred in x_pred:
    if aa % 100 == 0:
        print(aa)
    aa += 1
    t = [np.array(trame_pred)]
    predict_i = knn.predict(t)[0]
    predict_i = int(predict_i)
    if predict_i != i_NOKEY:
        res.append(d_list[predict_i])

print(res)

In [ ]:
for i_k in range(len(res)):
    if res[i_k] == 'CTRL' and res[i_k + 2] == 'SUPPR':
        print('##########')
        print(res[i_k:i_k + 22])
        

In [ ]:
print(len(res))

# KMEANS

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from random import randrange

dataset = get_dataset()
dlist = list(dataset)

data = dataset.items()
X = [i for i in range(17)]


for (key, value) in data:
    plt.scatter(X, value[0], label=key)
    
    values = []
    indice = 0
    for i in value:
        values.append([indice, i])
        indice = indice + 1
    
    kmean = Kmeans(n_clusters=17, random_state=randrange(200))
    y_pred = kmean.fit_predict(values)
    cc = np.transpose(kmean.cluster_centers_)
    
    plt.scatter(values[:, 0], values[:, 1], c=y_pred)
    plt.scatter(cc[0], cc[1], c='red')
    
    
plt.legend()
plt.show()
